# Class 10

In [2]:
struct MCDP_without_optimization
    u::Vector{Real}
    β::Real
    P::Matrix{Real}
end

In [42]:
job_search_problem = MCDP_without_optimization([1.0, 0.3], 1-0.05/12, [0.9 0.1; 0.03 0.97])

MCDP_without_optimization(Real[1.0, 0.3], 0.9958333333333333, Real[0.9 0.1; 0.03 0.97])

In [6]:
function iterate_value_function(v::Vector{T}, problem::MCDP_without_optimization) where T <: Real
    u, β, P = problem.u, problem.β, problem.P
    return u .+ β*P*v
end

iterate_value_function (generic function with 2 methods)

In [10]:
iterate_value_function([0.0, 0.0], job_search_problem)

2-element Vector{Float64}:
 1.0
 0.3

In [43]:
tolerance = 1e-6
maxiter = 100000

numiter = 1
v = zeros(2)
for t=1:maxiter
    new_v = iterate_value_function(v, job_search_problem)
    distance = sum((v .- new_v).^2)
    v = new_v
    if distance < tolerance 
        numiter = t
        break
    end
end
v

2-element Vector{Float64}:
 114.63042102531075
 109.39188033307501

In [38]:
numiter

47311

In [40]:
using LinearAlgebra: I
function solve(problem::MCDP_without_optimization)
    return (I - problem.β*problem.P) \ problem.u
end

solve (generic function with 1 method)

In [44]:
solve(job_search_problem)

2-element Vector{Float64}:
 114.79887745556647
 109.56033676333075

This is not needed:

~struct MCDP_with_discrete_choices
    choices::Vector{MCDP_without_optimization}
end~

In [46]:
low_intensity  = MCDP_without_optimization([1.0, 0.3], 0.95, [0.9 0.1; 0.01 0.99])
high_intensity = MCDP_without_optimization([1.0, 0.1], 0.95, [0.9 0.1; 0.3 0.7])
endogenous_job_search = [low_intensity, high_intensity]

2-element Vector{MCDP_without_optimization}:
 MCDP_without_optimization(Real[1.0, 0.3], 0.95, Real[0.9 0.1; 0.01 0.99])
 MCDP_without_optimization(Real[1.0, 0.1], 0.95, Real[0.9 0.1; 0.3 0.7])

In [61]:
function argmax_bellman(choices::Vector{MCDP_without_optimization}, v::Vector{T})::Vector{Int32} where T<:Real
    N = length(choices)
    K = length(choices[1].u)
    argmax = zeros(Int32, K)
    
    for k = 1:K
        best_index = 0
        best_value = -1e+16
        for n = 1:N
            candidate_value = choices[n].u + choices[n].β * choices[n].P * v
            if candidate_value[k] > best_value
                best_index = n
                best_value = candidate_value[k]
            end
        end
        argmax[k] = best_index
    end
    return argmax
end

argmax_bellman (generic function with 1 method)

In [59]:
function iterate_value_function(v::Vector{T}, choices::Vector{MCDP_without_optimization}) where T<:Real
    N = length(choices)
    K = length(choices[1].u)
    
    c = argmax_bellman(choices, v)
    
    new_v = zeros(K)
    for k = 1:K
        n = c[k]
        RHS = choices[n].u + choices[n].β * choices[n].P * v
        new_v[k] = RHS[k]
    end
    return new_v
end


iterate_value_function (generic function with 3 methods)

In [62]:
iterate_value_function([100.0, 0.0], endogenous_job_search)

2-element Vector{Float64}:
 86.5
 28.599999999999998

In [63]:
tolerance = 1e-6
maxiter = 100000

numiter = 1
v = zeros(2)
for t=1:maxiter
    new_v = iterate_value_function(v, endogenous_job_search)
    distance = sum((v .- new_v).^2)
    v = new_v
    if distance < tolerance 
        numiter = t
        break
    end
end
v

2-element Vector{Float64}:
 16.010234453917704
 13.917211198103743

# Policy function iteration

In [84]:
function iterate_policy_function(c::Vector{T}, choices::Vector{MCDP_without_optimization})::Vector{T} where T<:Integer
    u = similar(choices[1].u)
    P = similar(choices[1].P)
    β = choices[1].β
    
    for k = 1:length(c)
        choice = choices[c[k]]
        # for each choice c[k], what is the flow utility and the transition probs?
        u[k] = choice.u[k]
        P[k, :] .= choice.P[k, :]
    end
    # then solve for the value of this choice
    v = solve(MCDP_without_optimization(u, β, P))
    return argmax_bellman(choices, v)
end

iterate_policy_function (generic function with 1 method)

In [85]:
iterate_policy_function(ones(Int32, 2), endogenous_job_search)

2-element Vector{Int32}:
 1
 2

In [86]:
tolerance = 1e-6
maxiter = 100000

numiter = 1
c = ones(Int32, 2)
for t=1:maxiter
    new_c = iterate_policy_function(c, endogenous_job_search)
    distance = sum((c .- new_c).^2)
    c = new_c
    if distance < tolerance 
        numiter = t
        break
    end
end
c

2-element Vector{Int32}:
 1
 2